In [ ]:
import pandas             as pd
import numpy              as np
import regex              as re
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

In [ ]:
mri = pd.read_csv("../Data/stress_cardiac_mri.csv")

In [ ]:
print(f'The shape of the mri set is: {mri.shape}')

In [ ]:
mri.head(2)

In [ ]:
mri.columns

In [ ]:
mri = mri.rename({"Sex (1=male,0=female)": "Sex", "Hypertension (1=yes,0=no)": "Hypertension", "Name": "ID",
                  "History of smoking": "Smoker Status", "Tricuspid Regurgitation": "Tricusp Reg"}, axis = 1)
mri = mri.rename({"Aortic Regurgitation (0=none, 0.5 = trivial, 1=mild, 1.5=mild-moderate, 2=moderate, 2.5=moderate-severe, 3= severe)":
                  "Aortic Reg"}, axis = 1)
mri = mri.rename({"Mitral Regurgitation (0=none, 0.5 = trivial, 1=mild, 1.5=mild-moderate, 2=moderate, 2.5=moderate-severe, 3= severe)":
                  "Mitral Reg"}, axis = 1)

In [ ]:
mri.columns = mri.columns.str.replace(" ", "_")
mri.columns = mri.columns.str.lower()

In [ ]:
mri.columns

In [ ]:
mri.head(2)

In [ ]:
mri["diabetes"] = mri["diabetes"].apply(lambda x: 1 if x == "Yes" else 0)

In [ ]:
lv_wall_map         = {"Normal": 0, "MILD HYPERTROPHY": 1, "MODERATE HYPERTROPHY": 2, "SEVERE HYPERTROPHY": 3}
aortic_stenosis_map = {"None": 0, "Indeterminant": 0, "Trivial": 1, "Mild": 2, "MODERATE": 3, "SEVERE": 4}
tricuspid_map       = {"None": 0, "Trivial": 1, "MILD": 2, "MILD-MODERATE": 3, "MODERATE": 4, "MODERATE-SEVERE": 5, "SEVERE": 6}
aortic_mitral_map   = {0: 0, 0.5: 1, 1: 2, 1.5: 2, 2: 3, 2.5: 3, 3: 4}

In [ ]:
mri["lv_wall_thickness"] = mri["lv_wall_thickness"].map(lv_wall_map, na_action = "ignore")
mri["aortic_stenosis"]   = mri["aortic_stenosis"].map(aortic_stenosis_map, na_action = "ignore")
mri["tricusp_reg"]       = mri["tricusp_reg"].map(tricuspid_map)
mri["aortic_reg"]        = mri["aortic_reg"].map(aortic_mitral_map)
mri["mitral_reg"]        = mri["mitral_reg"].map(aortic_mitral_map)

In [ ]:
mri["smoker_status"].value_counts()

In [ ]:
mri["smoker_status"] = mri["smoker_status"].replace({"Unknown": "Never"})

In [ ]:
mri = pd.get_dummies(mri, columns = ["smoker_status"], drop_first = True )

In [ ]:
mri.columns = mri.columns.str.lower()

In [ ]:
mri.columns

In [ ]:
mri.isnull().mean()*100

In [ ]:
m = mri[(mri["sex"] == 1) & (mri["lvedv"] > 214)]
f = mri[(mri["sex"] == 0) & (mri["lvedv"] > 178)]

In [ ]:
print(f"There are {m.shape[0] + f.shape[0]} cases of left ventricular dilation")
print(f"The positive target makes up {(((m.shape[0] + f.shape[0]) / len(mri)) * 100)} percent of the data" )